<b>This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart?</b>

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses.


In [2]:
import time
x = time.time()

### Preprocessing the datasets

In the challenge, you are predicting item sales at stores in various locations for two 28-day time periods. Information about the data is found in the M5 Participants Guide.

#### Files
<b>calendar.csv</b> - Contains information about the dates on which the products are sold.

<b>sales_train_validation.csv</b> - Contains the historical daily unit sales data per product and store [d_1 - d_1913]

<b>sample_submission.csv</b> - The correct format for submissions. Reference the Evaluation tab for more info.

<b>sell_prices.csv</b> - Contains information about the price of the products sold per store and date.

<b>sales_train_evaluation.csv</b> - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)

### Engineering Features

<b>28 day rolling averages</b>  We will add the 28 day average of quantity_sold for each item within each state, store, and overall as three new features

When using for analysis with ML, the multiple  28-day period averages can be used for training. If we wanted to use a24-week period to determine latest trend for each ite, we will group data by item, then shift the averages by 28 days.

### 1. Import the relevant libraries

In [3]:
import pandas as pd
pd.set_option('max_rows', 999)
pd.set_option('max_columns', None)

import numpy as np

### 2.Save file paths as variables

In [1]:
#import sample_submission.csv from
sample_submission_path = 'data/sample_submission.csv'

#import sell_price.csv from
sell_prices_path = 'data/sell_prices.csv'

#import sales_train_validation.csv from
#sales_train_validation_path = 'data/sales_train_validation.csv'

#import sales_train_evaluation.csv from
sales_train_validation_path = 'data/sales_train_evaluation.csv'

#import calendar.csv from
calendar_path = 'data/calendar.csv'

#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file
final_preprocessed_file = 'm5_preprocessed_sales_train_validation.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file
validation_file = 'submission_kaggle_preprocessed_to_be_used_4prediction.csv'

#name of the preprocessed evaluation_data
evaluation_file = 'm5_preprocessed_evaluation_file.csv'

#name of the file with only training columns
training_file = 'days_1_1913_training_columns.csv'

### 2. Import and review the  files

While reviewing the data, I test the impact of downcasting the datatypes. An, while downcasting, I also add the datatypes to a 'schema' variable.

#### sample_submission.csv

In [46]:
sample_submission_df = pd.read_csv(sample_submission_path)
sample_submission_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<b>sample_submission</b> file doesn't contain the calendar, sales, and price information. Therefore, once the sales_train_validation data is preprocessed, I will extract that information and merge with the sample_submission data before making predictions.

Each row contains an id that is a concatenation of an item_id and a store_id, which is either validation (corresponding to the Public leaderboard), or evaluation (corresponding to the Private leaderboard). We are predicting 28 forecast days (F1-F28) of items sold for each row. 

For the validation rows, this corresponds to d_1914 - d_1941, and for the evaluation rows, this corresponds to d_1942 - d_1969. (Note: a month before the competition close, the ground truth for the validation rows will be provided.)

#### sell_price.csv

In [5]:
price_df = pd.read_csv(sell_prices_path)
price_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [6]:
#check datatypes
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [7]:
#downcast datatypes to reduce memory usage week is less than 32767. Therefore int16. 
price_df['wm_yr_wk'] = price_df['wm_yr_wk'].astype('int16')
price_df['sell_price'] = price_df['sell_price'].astype('float32',)
#check the new datatypes
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int16  
 3   sell_price  float32
dtypes: float32(1), int16(1), object(2)
memory usage: 143.5+ MB


It looks like sell_price.csv has the weekly prices for each item. To merge it with the sales_train_validation.csv, I will later perform a merge on 'store_id', 'item_id', 'wm_yr_wk' columns.

#### sales_train_validation.csv

In [8]:
#import sales_train data
#to check scoring in the public leaderboard, we need to use the sales_train_validation data
sales_train_validation = pd.read_csv(sales_train_validation_path)  #uncomment this line for validation
#sales_train_evaluation = pd.read_csv(sales_train_evaluation_path)   #uncomment this line for evaluation
sales_train_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [9]:
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


In [10]:
#check the maximum number of quantity sold in a day
max(sales_train_validation.describe().loc['max'])

763.0

In [11]:
#max quantity sold is 763. Therefore, we can downcast
#int64 to int8
for col in sales_train_validation.select_dtypes('int64').columns:
    sales_train_validation[col] = sales_train_validation[col].astype('int16')

#check the new datatypes
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


Memory usage went down signifantly.

#### calendar.csv

In [12]:
#importing the calendar_df.csv
calendar_df = pd.read_csv(calendar_path, parse_dates=['date'])
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [13]:
#Check columns, memory usage, and datatypes
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


<b>Calendar.csv</b> file has the characteristics of each day. The information is stored as a text in categorical column. These must be encoded and then merged with the sales data.

## Preprocess the competition files

### 3. Process the calendar file

In [14]:
#importing the calendar_df.csv
calendar_df = pd.read_csv(calendar_path)

#int64 to int16
for col in calendar_df.select_dtypes('int64').columns:
    calendar_df[col] = calendar_df[col].astype('int16')

# Identify the non-numerical columns
categorical_columns_calendar_df = calendar_df.select_dtypes('object').copy()

# Split the 'd' values and extract the day number
calendar_df['day'] = categorical_columns_calendar_df['d'].map(lambda x: x.split('d_')[1] ).astype('int16')

# Identify the unique values of all the event Names
unique_event_names = []
for col in ['event_name_1', 'event_name_2']:
    unique_event_names.extend(categorical_columns_calendar_df[col].unique())
    
# Make a list a of the unique values
unique_event_names = list(set(unique_event_names))

#Remove np.nan values
unique_event_names.remove(np.nan)

# Identify the unique values of all the event Types
unique_event_types = []
for col in ['event_type_1', 'event_type_2']:
    unique_event_types.extend(categorical_columns_calendar_df[col].unique())

# Make a list a of the unique values
unique_event_types = list(set(unique_event_types))

# Remove the np.nan values
unique_event_types.remove(np.nan)

#add the unique event types and names as headers and fill null values with 0
calendar_df = calendar_df.reindex(columns=list(calendar_df.columns)+unique_event_names+ unique_event_types).fillna(0)

# If there is an event on a day, add 1 to the cell under the column of that event
for col in ['event_name_1','event_type_1', 'event_name_2', 'event_type_2']:  #choose the first column
        for indx, event in enumerate(calendar_df[calendar_df[col]!=0][col]):  #choose non_zero the cells 
            calendar_df[event][indx] = 1  #find the column with the event name and assign 1 at the right index
            calendar_df[event] = calendar_df[event].astype('int8')
            
#drop event_name and event_type columns
calendar_df.drop(['date', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2','event_type_2'], axis=1, inplace=True )

calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   wm_yr_wk             1969 non-null   int16 
 1   wday                 1969 non-null   int16 
 2   month                1969 non-null   int16 
 3   year                 1969 non-null   int16 
 4   d                    1969 non-null   object
 5   snap_CA              1969 non-null   int16 
 6   snap_TX              1969 non-null   int16 
 7   snap_WI              1969 non-null   int16 
 8   day                  1969 non-null   int16 
 9   VeteransDay          1969 non-null   int8  
 10  ValentinesDay        1969 non-null   int8  
 11  ColumbusDay          1969 non-null   int8  
 12  NBAFinalsEnd         1969 non-null   int8  
 13  StPatricksDay        1969 non-null   int8  
 14  Father's day         1969 non-null   int8  
 15  Ramadan starts       1969 non-null   int8  
 16  LentSt

C:\Users\galip\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 4. Process the sales data

In [15]:
#Let's remember how the sales data looked like
sales_train_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [16]:
#Check the datatypes and columns in sales data
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [17]:
#Encode dept_id, cat_id, state_id and store_id
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['dept_id'],drop_first=True)], axis=1)
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['cat_id'],drop_first=True)], axis=1)
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['state_id'],drop_first=True)], axis=1)
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['store_id'],drop_first=True)], axis=1)

In [18]:
#Save the non-sales column names
sales_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

In [19]:
#melt the sales data on ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
#then merge with calendar on ['d'], and
#merge with price data on ['store_id','item_id','wm_yr_wk']
#and export the merged data into the output folder to free up memory

sales_train_validation_processing = \
sales_train_validation.melt(id_vars=sales_cols, value_name='quantity_sold', var_name='d')\
.merge(calendar_df, on=['d'], how='left').merge(price_df, on=['store_id','item_id','wm_yr_wk'], how='left')

#Review the processed file
sales_train_validation_processing

,id,item_id,dept_id,cat_id,store_id,state_id,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,d,quantity_sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,day,VeteransDay,ValentinesDay,ColumbusDay,NBAFinalsEnd,StPatricksDay,Father's day,Ramadan starts,LentStart,Purim End,Christmas,NewYear,Halloween,OrthodoxEaster,IndependenceDay,Cinco De Mayo,LaborDay,SuperBowl,Mother's day,MemorialDay,Eid al-Fitr,Thanksgiving,Easter,OrthodoxChristmas,EidAlAdha,Pesach End,LentWeek2,PresidentsDay,NBAFinalsStart,MartinLutherKingDay,Chanukah End,Cultural,Religious,Sporting,National,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,1,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.98
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.48
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,2,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.98
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.28


## Feature Engineering

In [20]:
#get the unique item_ids
item_ids = list(sales_train_validation.item_id)

#### As a new feature, we can add average weekly or monthly sales to the sales_train_validation_processing df.

In [21]:
###Calculate average item sales per state
#Group the temporary dataframe by state_id and item_id and day to find the weekly  sale averages in each state per item,
weekly_state_avg = sales_train_validation_processing[['state_id', 'item_id','wm_yr_wk', 'quantity_sold']].groupby(by=['state_id', 'item_id','wm_yr_wk']).mean()

weekly_state_avg.head()

quantity_sold
state_id item_id     wm_yr_wk               
CA       FOODS_1_001 11101          1.214286
                     11102          1.071429
                     11103          1.500000
                     11104          1.714286
                     11105          1.428571

#### First, we will add 4 to the wm_yr_wk column prior to merging with the main df to align an item with its average sales data from 4 weeks ago.

In [22]:
#reset the index
weekly_state_avg = weekly_state_avg.reset_index()

#rename the columns
weekly_state_avg.columns = ['state_id', 'item_id','wm_yr_wk', 'weekly_avg_4weeks_ago']

#shift the 'wm_yr_wk' column by 4
weekly_state_avg['wm_yr_wk'] = [x+4 if int(str(x)[-2:])<49 else x+52 for x in weekly_state_avg.wm_yr_wk]

#Review the output
weekly_state_avg.head(110)

,state_id,item_id,wm_yr_wk,weekly_avg_4weeks_ago
0,CA,FOODS_1_001,11105,1.214286
1,CA,FOODS_1_001,11106,1.071429
2,CA,FOODS_1_001,11107,1.500000
3,CA,FOODS_1_001,11108,1.714286
4,CA,FOODS_1_001,11109,1.428571
5,CA,FOODS_1_001,11110,1.571429
6,CA,FOODS_1_001,11111,1.000000
7,CA,FOODS_1_001,11112,1.035714
8,CA,FOODS_1_001,11113,0.892857
9,CA,FOODS_1_001,11114,1.178571


#### Now, it is time to merge this data with the sales_train_validation_processing df

In [23]:
# The latest version of the sales_train_validation is sales_train_validation_processing df
# We will left merge the weekly average data on ['state_id,'item_id','wm_yr_wk']
sales_train_validation_processing = sales_train_validation_processing.merge(weekly_state_avg, how='left' )

#Review the output
sales_train_validation_processing

,id,item_id,dept_id,cat_id,store_id,state_id,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,d,quantity_sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,day,VeteransDay,ValentinesDay,ColumbusDay,NBAFinalsEnd,StPatricksDay,Father's day,Ramadan starts,LentStart,Purim End,Christmas,NewYear,Halloween,OrthodoxEaster,IndependenceDay,Cinco De Mayo,LaborDay,SuperBowl,Mother's day,MemorialDay,Eid al-Fitr,Thanksgiving,Easter,OrthodoxChristmas,EidAlAdha,Pesach End,LentWeek2,PresidentsDay,NBAFinalsStart,MartinLutherKingDay,Chanukah End,Cultural,Religious,Sporting,National,sell_price,weekly_avg_4weeks_ago
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394515,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,1,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.98,0.619048
59394516,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.48,0.666667
59394517,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,2,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.98,0.714286
59394518,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.28,1.952381


#### Next, we will add 1 to the month column of the monthly_state_avg df prior to merging with the main df to align each item with its previous month average sales data

In [24]:
#Group the temporary dataframe by state_id and item_id and day to find the  monthly sale averages in each state per item,
monthly_state_avg = sales_train_validation_processing[['state_id', 'item_id','year','month', 'quantity_sold']].groupby(by=['state_id', 'item_id','year','month']).mean()

#Review the output
monthly_state_avg

quantity_sold
state_id item_id         year month               
CA       FOODS_1_001     2011 1           0.916667
                              2           1.437500
                              3           1.161290
                              4           1.116667
                              5           1.604839
...                                            ...
WI       HOUSEHOLD_2_516 2016 1           0.086022
                              2           0.091954
                              3           0.053763
                              4           0.055556
                              5           0.090909

[594555 rows x 1 columns]

In [25]:
#Adding 1 to the month count to align each item with sales average from previous month
monthly_state_avg.index = pd.MultiIndex.from_tuples([(x[0], x[1],x[2], x[3]+1) if x[3]<12 else (x[0], x[1],x[2]+1, 1) for x in monthly_state_avg.index ])

#reset the index
monthly_state_avg = monthly_state_avg.reset_index()

#rename the columns
monthly_state_avg.columns = ['state_id', 'item_id','year','month', 'last_month_avg']

#The months should be starting from 2011 February now.
monthly_state_avg

,state_id,item_id,year,month,last_month_avg
0,CA,FOODS_1_001,2011,2,0.916667
1,CA,FOODS_1_001,2011,3,1.437500
2,CA,FOODS_1_001,2011,4,1.161290
3,CA,FOODS_1_001,2011,5,1.116667
4,CA,FOODS_1_001,2011,6,1.604839
...,...,...,...,...,...
594550,WI,HOUSEHOLD_2_516,2016,2,0.086022
594551,WI,HOUSEHOLD_2_516,2016,3,0.091954
594552,WI,HOUSEHOLD_2_516,2016,4,0.053763
594553,WI,HOUSEHOLD_2_516,2016,5,0.055556


#### Now it is time to merge previous month and weekly average from 28 days ago with the sales_train_validation df

In [26]:
# The latest version of the sales_train_validation is sales_train_validation_processing df
# We will left merge the monthly average data on ['state_id,'year','month']
sales_train_validation_processing = sales_train_validation_processing.merge(monthly_state_avg, how='left' )

#Review the output
sales_train_validation_processing

,id,item_id,dept_id,cat_id,store_id,state_id,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,d,quantity_sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,day,VeteransDay,ValentinesDay,ColumbusDay,NBAFinalsEnd,StPatricksDay,Father's day,Ramadan starts,LentStart,Purim End,Christmas,NewYear,Halloween,OrthodoxEaster,IndependenceDay,Cinco De Mayo,LaborDay,SuperBowl,Mother's day,MemorialDay,Eid al-Fitr,Thanksgiving,Easter,OrthodoxChristmas,EidAlAdha,Pesach End,LentWeek2,PresidentsDay,NBAFinalsStart,MartinLutherKingDay,Chanukah End,Cultural,Religious,Sporting,National,sell_price,weekly_avg_4weeks_ago,last_month_avg
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394515,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,1,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.98,0.619048,0.333333
59394516,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.48,0.666667,0.522222
59394517,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,2,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.98,0.714286,1.377778
59394518,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.28,1.952381,1.388889


In [27]:
#save the datatypes as a variable
schema = dict(zip(sales_train_validation_processing.columns, sales_train_validation_processing.dtypes))

In [28]:
organized_cols = [
         'id',
         'd', 'day', 

         'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

In [29]:
#Reorganize the columns
sales_train_validation_processing = sales_train_validation_processing[organized_cols]

## Export the final preprocessed file

In [6]:
schema={
 'FOODS_2': 'uint8', 'FOODS_3': 'uint8', 'HOBBIES_1': 'uint8', 'HOBBIES_2': 'uint8', 'HOUSEHOLD_1': 'uint8',
 'HOUSEHOLD_2': 'uint8', 'HOBBIES': 'uint8', 'HOUSEHOLD': 'uint8', 'TX': 'uint8', 'WI': 'uint8', 'CA_2': 'uint8',
 'CA_3': 'uint8','CA_4': 'uint8', 'TX_1': 'uint8', 'TX_2': 'uint8', 'TX_3': 'uint8', 'WI_1': 'uint8', 'WI_2': 'uint8',
 'WI_3': 'uint8', 'd': 'O', 'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int16', 'month': 'int16',
 'year': 'int16', 'snap_CA': 'int16', 'snap_TX': 'int16', 'snap_WI': 'int16', 'day': 'int16', 'Cinco De Mayo': 'int8',
 'OrthodoxChristmas': 'int8', "Father's day": 'int8', 'Thanksgiving': 'int8', "Mother's day": 'int8', 'NBAFinalsStart': 'int8',
 'Christmas': 'int8', 'Chanukah End': 'int8', 'EidAlAdha': 'int8', 'MemorialDay': 'int8', 'Halloween': 'int8',
 'VeteransDay': 'int8','SuperBowl': 'int8', 'LentStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
 'LaborDay': 'int8', 'Easter': 'int8', 'NBAFinalsEnd': 'int8', 'Purim End': 'int8', 'IndependenceDay': 'int8',
 'Ramadan starts': 'int8', 'StPatricksDay': 'int8', 'LentWeek2': 'int8', 'OrthodoxEaster': 'int8', 'Pesach End': 'int8',
 'NewYear': 'int8', 'PresidentsDay': 'int8', 'Eid al-Fitr': 'int8', 'ColumbusDay': 'int8', 'Sporting': 'int8',
 'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'sell_price': 'float16', 'last_month_avg': 'float16',
 'weekly_avg_4weeks_ago': 'float16'}

In [32]:
sales_train_validation_processing.to_csv(export_path+final_preprocessed_file)

In [56]:
del(sales_train_validation_processing)

### Identify the training columns

In [21]:
training_cols = ['id','item_id','day', 
                 'wm_yr_wk', 'year', 'FOODS_2', 'FOODS_3', 'HOBBIES_1',
       'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 'TX',
       'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2',
       'WI_3', 'snap_CA', 'snap_TX', 'snap_WI', 'Cinco De Mayo',
       'OrthodoxChristmas', 'Father\'s day', 'Thanksgiving', 'Mother\'s day',
       'NBAFinalsStart', 'Christmas', 'Chanukah End', 'EidAlAdha',
       'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart',
       'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter',
       'NBAFinalsEnd', 'Purim End', 'IndependenceDay', 'Ramadan starts',
       'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear',
       'PresidentsDay', 'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural',
       'National', 'Religious', 'wday', 'month', 'sell_price',
       'last_month_avg', 'weekly_avg_4weeks_ago', 'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]

In [22]:
#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]

###  Format the submission / validation file

In [6]:
#Reading the sample submission data to prepare for predictions
submission_file = pd.read_csv(sample_submission_path)

day_cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

#set the first day in the training data
first_day = 1 # training_data.day_indx[0]

#set the last day in the training data
last_day = 1941 #training_data.day_indx[-1]

#set the last 28 days in the training data
next_28_days = list(range(first_day,last_day+1))[-28:]

#rename the col names of the submisson file to last 28 days
submission_file.rename(columns=dict(zip(list(day_cols), next_28_days)), inplace=True)


#add item_id
submission_file['item_id'] = submission_file.id.apply(lambda x: x.split('_')[0]+'_'+ x.split('_')[1]+ '_'+ x.split('_')[2])

# add store_id
submission_file['store_id'] = submission_file.id.apply(lambda x: x.split('_')[3]+'_'+x.split('_')[4])

#melt the file
submission_file = submission_file.melt(id_vars=['id','item_id','store_id'], var_name='day', value_name='quantity_sold')


In [53]:
#Review the submission file so far
submission_file

,id,item_id,store_id,day,quantity_sold
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,1914,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,1914,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,1914,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,CA_1,1914,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,CA_1,1914,0
...,...,...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,FOODS_3_823,WI_3,1941,0
1707436,FOODS_3_824_WI_3_evaluation,FOODS_3_824,WI_3,1941,0
1707437,FOODS_3_825_WI_3_evaluation,FOODS_3_825,WI_3,1941,0
1707438,FOODS_3_826_WI_3_evaluation,FOODS_3_826,WI_3,1941,0


In [14]:
#Now merge the submission file with the processed sales_train_evaluation file to extract sales, calendar, price data

#read the processed data in chunks to free up memory
sales_train_validation_processing = pd.read_csv(export_path+final_preprocessed_file, dtype=schema, index_col=0 , chunksize=1000000)

validation_df = pd.DataFrame()

#read the sales_train_validation_process

for chunk in sales_train_validation_processing:
    
    #Extract the quantity sold
    validation_df = pd.concat( [validation_df, submission_file.merge(chunk, on=['id','item_id','store_id','day'], how='inner', suffixes=['','_y'])])
    
validation_df

,id_pred,item_id,store_id,day,quantity_sold_pred,id,d,dept_id,cat_id,state_id,wm_yr_wk,year,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,snap_CA,snap_TX,snap_WI,Cinco De Mayo,OrthodoxChristmas,Father's day,Thanksgiving,Mother's day,NBAFinalsStart,Christmas,Chanukah End,EidAlAdha,MemorialDay,Halloween,VeteransDay,SuperBowl,LentStart,MartinLutherKingDay,ValentinesDay,LaborDay,Easter,NBAFinalsEnd,Purim End,IndependenceDay,Ramadan starts,StPatricksDay,LentWeek2,OrthodoxEaster,Pesach End,NewYear,PresidentsDay,Eid al-Fitr,ColumbusDay,Sporting,Cultural,National,Religious,wday,month,sell_price,last_month_avg,weekly_avg_4weeks_ago,quantity_sold
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,1914,0,HOBBIES_1_001_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,11613,2016,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,8.382812,0.991699,0.928711,0
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,CA_1,1914,0,HOBBIES_1_001_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,11613,2016,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,8.382812,0.991699,0.928711,0
2,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,1914,0,HOBBIES_1_002_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,11613,2016,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,3.970703,0.137085,0.107117,0
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,CA_1,1914,0,HOBBIES_1_002_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,11613,2016,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,3.970703,0.137085,0.107117,0
4,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,1914,0,HOBBIES_1_003_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,11613,2016,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,2.970703,0.379150,0.285645,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789035,FOODS_3_825_WI_3_evaluation,FOODS_3_825,WI_3,1941,0,FOODS_3_825_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,11617,2016,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,3.980469,1.377930,0.714355,2
789036,FOODS_3_826_WI_3_validation,FOODS_3_826,WI_3,1941,0,FOODS_3_826_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,11617,2016,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,1.280273,1.388672,1.952148,0
789037,FOODS_3_826_WI_3_evaluation,FOODS_3_826,WI_3,1941,0,FOODS_3_826_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,11617,2016,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,1.280273,1.388672,1.952148,0
789038,FOODS_3_827_WI_3_validation,FOODS_3_827,WI_3,1941,0,FOODS_3_827_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,11617,2016,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,1.000000,2.166016,1.809570,1


In [15]:
#Export the processed submission file
validation_df.to_csv(export_path + validation_file)

### Check run time

In [16]:
y= time.time()
print(f'It took {(y-x)/60} minutes to complete.')

It took 22.52691849072774 minutes to complete.


# BUILD MODELS and PREDICT

### Prepare the training and validation dataframes

In [23]:
#select non-null rows and training columns from the sales_train_validation_process data
training_df = pd.read_csv(export_path+final_preprocessed_file, usecols=training_cols, dtype=schema)
validation_df = pd.read_csv(export_path+validation_file, dtype=schema)

In [24]:
#filter the train data for days starting from 29 to 1913
training_df = training_df.loc[(28<training_df.day)&(training_df.day<1914)]

In [25]:
#drop missing values
training_df.dropna(subset=['sell_price','weekly_avg_4weeks_ago'], inplace=True)

In [26]:
training_df.isna().any()

id                       False
day                      False
item_id                  False
wm_yr_wk                 False
year                     False
FOODS_2                  False
FOODS_3                  False
HOBBIES_1                False
HOBBIES_2                False
HOUSEHOLD_1              False
HOUSEHOLD_2              False
HOBBIES                  False
HOUSEHOLD                False
TX                       False
WI                       False
CA_2                     False
CA_3                     False
CA_4                     False
TX_1                     False
TX_2                     False
TX_3                     False
WI_1                     False
WI_2                     False
WI_3                     False
snap_CA                  False
snap_TX                  False
snap_WI                  False
Cinco De Mayo            False
OrthodoxChristmas        False
Father's day             False
Thanksgiving             False
Mother's day             False
NBAFinal

In [13]:
validation_df.isna().any()

Unnamed: 0               False
id_pred                  False
item_id                  False
store_id                 False
day                      False
quantity_sold_pred       False
id                       False
d                        False
dept_id                  False
cat_id                   False
state_id                 False
wm_yr_wk                 False
year                     False
FOODS_2                  False
FOODS_3                  False
HOBBIES_1                False
HOBBIES_2                False
HOUSEHOLD_1              False
HOUSEHOLD_2              False
HOBBIES                  False
HOUSEHOLD                False
TX                       False
WI                       False
CA_2                     False
CA_3                     False
CA_4                     False
TX_1                     False
TX_2                     False
TX_3                     False
WI_1                     False
WI_2                     False
WI_3                     False
snap_CA 

## MODEL 1: SGD ML Multiple Linear Regression

#### This is the baseline Gradient Descent Machine learning model for multiple linear regression

In [87]:
x=time.time()  #record the start time

weight_dict = {}  #this will store the best weights for each item

#make a list of the unique item_ids
items = validation_df.item_id.unique()

#Set parameters
epochs = 50
early_stop_threshold = 5
learning_rate = 0.0001
alpha = 0.01

for item in items:  #calculate the weights for each item by iterating over each item
    
    
    temp_df = training_df[training_df['item_id'] == item]

    X = temp_df[input_cols]   #filter the dataset on the item_id

    target = np.array(temp_df[target_col]).reshape(-1,1)   #labels



    #initialize coefficients and the bias
    np.random.seed(10)
    W = np.random.rand(61, 1)/60
    B = np.random.random(size=(1))/100
    N = np.random.rand(X.shape[0],1)/30

    #count the number of items
    num_observations = X.shape[0]

    #initialize the loss and and a loss list to help with early stopping
    losses = [0]
    loss_score = 0


    for epoch in range(epochs):

        Y =  X.dot(W) +  B + N #calculate the targets

        deltas = target - Y  #Find the error

        loss = deltas.pow(2).mean().pow(0.5)   #Loss function
        
        #set up an early stopping mechanism
        #add the loss to the losses list
        
        if losses[-1] < loss[0]:

            loss_score += 1   #keep track of the change in loss function
            
            losses.append(loss[0])


        print(f'Epoch: {epoch}  Loss: {loss[0]:.3f}')


        if loss_score > early_stop_threshold or epoch == (epochs-1):   #If the loss is increasing  or ran out of epochs stop learning

            weight_dict[item] = (W,B)
            
            
            #prepare a submission file with the validation file
            
            result = validation_df[validation_df['item_id'] == item][['id', 'item_id', 'day', 'quantity_sold']]   #create a new dataframe to save predictions

            result['prediction'] =  validation_df[validation_df.item_id==item][input_cols].dot(weight_dict[item][0])+weight_dict[item][1]#save the predictions

            result.to_csv(export_path + 'submission_file_output_ML_Multiple_Reg_model_1.csv', mode='a', header=False)              

            break

        #scale the deltas
        deltas_scaled = deltas / num_observations

        #Optimize
        #update the weights biases
        w_momentum = alpha * learning_rate * np.dot(X.T, deltas_scaled)
        W = W - learning_rate * np.dot(X.T, deltas_scaled) - w_momentum
        
        b_momentum = alpha * learning_rate * np.sum(deltas_scaled)
        B = B - learning_rate * np.sum(deltas_scaled) - b_momentum
        
        
        
y= time.time()
print(f'It took {(y-x)/60} minutes to complete.')


#pickle the weights
import pickle
with open('M5_Multiple_Linear_Regression_Baseline_Weights_Dictionary.pickle', 'wb') as f:
    pickle.dump(weight_dict, f)

Epoch: 0  Loss: 0.754
Epoch: 1  Loss: 0.754
Epoch: 2  Loss: 0.754
Epoch: 3  Loss: 0.754
Epoch: 4  Loss: 0.754
Epoch: 5  Loss: 0.755
Epoch: 0  Loss: 0.604
Epoch: 1  Loss: 0.604
Epoch: 2  Loss: 0.604
Epoch: 3  Loss: 0.604
Epoch: 4  Loss: 0.604
Epoch: 5  Loss: 0.604
Epoch: 0  Loss: 0.491
Epoch: 1  Loss: 0.491
Epoch: 2  Loss: 0.491
Epoch: 3  Loss: 0.491
Epoch: 4  Loss: 0.491
Epoch: 5  Loss: 0.491
Epoch: 0  Loss: 3.223
Epoch: 1  Loss: 3.232
Epoch: 2  Loss: 3.241
Epoch: 3  Loss: 3.251
Epoch: 4  Loss: 3.260
Epoch: 5  Loss: 3.270
Epoch: 0  Loss: 1.377
Epoch: 1  Loss: 1.379
Epoch: 2  Loss: 1.381
Epoch: 3  Loss: 1.383
Epoch: 4  Loss: 1.385
Epoch: 5  Loss: 1.387
Epoch: 0  Loss: 1.779
Epoch: 1  Loss: 1.781
Epoch: 2  Loss: 1.782
Epoch: 3  Loss: 1.784
Epoch: 4  Loss: 1.785
Epoch: 5  Loss: 1.787
Epoch: 0  Loss: 0.497
Epoch: 1  Loss: 0.497
Epoch: 2  Loss: 0.498
Epoch: 3  Loss: 0.498
Epoch: 4  Loss: 0.498
Epoch: 5  Loss: 0.498
Epoch: 0  Loss: 8.395
Epoch: 1  Loss: 8.425
Epoch: 2  Loss: 8.454
Epoch: 3  

Epoch: 0  Loss: 0.364
Epoch: 1  Loss: 0.364
Epoch: 2  Loss: 0.365
Epoch: 3  Loss: 0.365
Epoch: 4  Loss: 0.366
Epoch: 5  Loss: 0.367
Epoch: 0  Loss: 1.067
Epoch: 1  Loss: 1.067
Epoch: 2  Loss: 1.067
Epoch: 3  Loss: 1.067
Epoch: 4  Loss: 1.067
Epoch: 5  Loss: 1.067
Epoch: 0  Loss: 7.459
Epoch: 1  Loss: 7.472
Epoch: 2  Loss: 7.486
Epoch: 3  Loss: 7.500
Epoch: 4  Loss: 7.513
Epoch: 5  Loss: 7.528
Epoch: 0  Loss: 0.902
Epoch: 1  Loss: 0.903
Epoch: 2  Loss: 0.904
Epoch: 3  Loss: 0.904
Epoch: 4  Loss: 0.905
Epoch: 5  Loss: 0.906
Epoch: 0  Loss: 0.540
Epoch: 1  Loss: 0.541
Epoch: 2  Loss: 0.541
Epoch: 3  Loss: 0.542
Epoch: 4  Loss: 0.544
Epoch: 5  Loss: 0.545
Epoch: 0  Loss: 0.381
Epoch: 1  Loss: 0.382
Epoch: 2  Loss: 0.382
Epoch: 3  Loss: 0.383
Epoch: 4  Loss: 0.384
Epoch: 5  Loss: 0.384
Epoch: 0  Loss: 1.029
Epoch: 1  Loss: 1.030
Epoch: 2  Loss: 1.031
Epoch: 3  Loss: 1.032
Epoch: 4  Loss: 1.033
Epoch: 5  Loss: 1.034
Epoch: 0  Loss: 0.902
Epoch: 1  Loss: 0.903
Epoch: 2  Loss: 0.904
Epoch: 3  

Epoch: 0  Loss: 3.388
Epoch: 1  Loss: 3.392
Epoch: 2  Loss: 3.396
Epoch: 3  Loss: 3.400
Epoch: 4  Loss: 3.404
Epoch: 5  Loss: 3.408
Epoch: 0  Loss: 1.044
Epoch: 1  Loss: 1.044
Epoch: 2  Loss: 1.045
Epoch: 3  Loss: 1.046
Epoch: 4  Loss: 1.047
Epoch: 5  Loss: 1.048
Epoch: 0  Loss: 0.602
Epoch: 1  Loss: 0.602
Epoch: 2  Loss: 0.603
Epoch: 3  Loss: 0.603
Epoch: 4  Loss: 0.603
Epoch: 5  Loss: 0.603
Epoch: 0  Loss: 5.001
Epoch: 1  Loss: 5.020
Epoch: 2  Loss: 5.040
Epoch: 3  Loss: 5.060
Epoch: 4  Loss: 5.081
Epoch: 5  Loss: 5.101
Epoch: 0  Loss: 0.547
Epoch: 1  Loss: 0.552
Epoch: 2  Loss: 0.557
Epoch: 3  Loss: 0.562
Epoch: 4  Loss: 0.568
Epoch: 5  Loss: 0.575
Epoch: 0  Loss: 0.346
Epoch: 1  Loss: 0.347
Epoch: 2  Loss: 0.347
Epoch: 3  Loss: 0.348
Epoch: 4  Loss: 0.349
Epoch: 5  Loss: 0.350
Epoch: 0  Loss: 0.722
Epoch: 1  Loss: 0.722
Epoch: 2  Loss: 0.722
Epoch: 3  Loss: 0.722
Epoch: 4  Loss: 0.722
Epoch: 5  Loss: 0.722
Epoch: 0  Loss: 0.480
Epoch: 1  Loss: 0.481
Epoch: 2  Loss: 0.481
Epoch: 3  

Epoch: 0  Loss: 0.641
Epoch: 1  Loss: 0.641
Epoch: 2  Loss: 0.641
Epoch: 3  Loss: 0.641
Epoch: 4  Loss: 0.641
Epoch: 5  Loss: 0.641
Epoch: 0  Loss: 0.458
Epoch: 1  Loss: 0.460
Epoch: 2  Loss: 0.462
Epoch: 3  Loss: 0.464
Epoch: 4  Loss: 0.467
Epoch: 5  Loss: 0.469
Epoch: 0  Loss: 1.176
Epoch: 1  Loss: 1.177
Epoch: 2  Loss: 1.178
Epoch: 3  Loss: 1.178
Epoch: 4  Loss: 1.179
Epoch: 5  Loss: 1.180
Epoch: 0  Loss: 0.789
Epoch: 1  Loss: 0.789
Epoch: 2  Loss: 0.789
Epoch: 3  Loss: 0.790
Epoch: 4  Loss: 0.790
Epoch: 5  Loss: 0.790
Epoch: 0  Loss: 0.921
Epoch: 1  Loss: 0.923
Epoch: 2  Loss: 0.924
Epoch: 3  Loss: 0.926
Epoch: 4  Loss: 0.927
Epoch: 5  Loss: 0.929
Epoch: 0  Loss: 1.681
Epoch: 1  Loss: 1.686
Epoch: 2  Loss: 1.691
Epoch: 3  Loss: 1.696
Epoch: 4  Loss: 1.700
Epoch: 5  Loss: 1.706
Epoch: 0  Loss: 1.985
Epoch: 1  Loss: 1.987
Epoch: 2  Loss: 1.989
Epoch: 3  Loss: 1.991
Epoch: 4  Loss: 1.992
Epoch: 5  Loss: 1.994
Epoch: 0  Loss: 0.732
Epoch: 1  Loss: 0.732
Epoch: 2  Loss: 0.732
Epoch: 3  

Epoch: 0  Loss: 0.861
Epoch: 1  Loss: 0.861
Epoch: 2  Loss: 0.862
Epoch: 3  Loss: 0.863
Epoch: 4  Loss: 0.864
Epoch: 5  Loss: 0.865
Epoch: 0  Loss: 4.826
Epoch: 1  Loss: 4.832
Epoch: 2  Loss: 4.837
Epoch: 3  Loss: 4.843
Epoch: 4  Loss: 4.848
Epoch: 5  Loss: 4.854
Epoch: 0  Loss: 0.689
Epoch: 1  Loss: 0.689
Epoch: 2  Loss: 0.689
Epoch: 3  Loss: 0.690
Epoch: 4  Loss: 0.690
Epoch: 5  Loss: 0.690
Epoch: 0  Loss: 0.384
Epoch: 1  Loss: 0.385
Epoch: 2  Loss: 0.385
Epoch: 3  Loss: 0.386
Epoch: 4  Loss: 0.386
Epoch: 5  Loss: 0.387
Epoch: 0  Loss: 1.020
Epoch: 1  Loss: 1.021
Epoch: 2  Loss: 1.022
Epoch: 3  Loss: 1.023
Epoch: 4  Loss: 1.023
Epoch: 5  Loss: 1.024
Epoch: 0  Loss: 0.640
Epoch: 1  Loss: 0.640
Epoch: 2  Loss: 0.640
Epoch: 3  Loss: 0.640
Epoch: 4  Loss: 0.640
Epoch: 5  Loss: 0.640
Epoch: 0  Loss: 1.048
Epoch: 1  Loss: 1.049
Epoch: 2  Loss: 1.049
Epoch: 3  Loss: 1.050
Epoch: 4  Loss: 1.051
Epoch: 5  Loss: 1.051
Epoch: 0  Loss: 0.416
Epoch: 1  Loss: 0.416
Epoch: 2  Loss: 0.417
Epoch: 3  

...

Epoch: 0  Loss: 0.867
Epoch: 1  Loss: 0.867
Epoch: 2  Loss: 0.868
Epoch: 3  Loss: 0.869
Epoch: 4  Loss: 0.869
Epoch: 5  Loss: 0.870
Epoch: 0  Loss: 4.932
Epoch: 1  Loss: 4.955
Epoch: 2  Loss: 4.979
Epoch: 3  Loss: 5.003
Epoch: 4  Loss: 5.027
Epoch: 5  Loss: 5.051
Epoch: 0  Loss: 1.250
Epoch: 1  Loss: 1.251
Epoch: 2  Loss: 1.252
Epoch: 3  Loss: 1.252
Epoch: 4  Loss: 1.253
Epoch: 5  Loss: 1.254
Epoch: 0  Loss: 3.206
Epoch: 1  Loss: 3.219
Epoch: 2  Loss: 3.232
Epoch: 3  Loss: 3.245
Epoch: 4  Loss: 3.259
Epoch: 5  Loss: 3.273
Epoch: 0  Loss: 1.923
Epoch: 1  Loss: 1.929
Epoch: 2  Loss: 1.935
Epoch: 3  Loss: 1.940
Epoch: 4  Loss: 1.946
Epoch: 5  Loss: 1.952
Epoch: 0  Loss: 3.301
Epoch: 1  Loss: 3.311
Epoch: 2  Loss: 3.321
Epoch: 3  Loss: 3.331
Epoch: 4  Loss: 3.342
Epoch: 5  Loss: 3.352
Epoch: 0  Loss: 1.602
Epoch: 1  Loss: 1.606
Epoch: 2  Loss: 1.611
Epoch: 3  Loss: 1.616
Epoch: 4  Loss: 1.621
Epoch: 5  Loss: 1.625
Epoch: 0  Loss: 1.231
Epoch: 1  Loss: 1.232
Epoch: 2  Loss: 1.233
Epoch: 3  

Epoch: 0  Loss: 1.457
Epoch: 1  Loss: 1.459
Epoch: 2  Loss: 1.461
Epoch: 3  Loss: 1.463
Epoch: 4  Loss: 1.465
Epoch: 5  Loss: 1.467
Epoch: 0  Loss: 2.691
Epoch: 1  Loss: 2.701
Epoch: 2  Loss: 2.712
Epoch: 3  Loss: 2.722
Epoch: 4  Loss: 2.733
Epoch: 5  Loss: 2.745
Epoch: 0  Loss: 0.942
Epoch: 1  Loss: 0.943
Epoch: 2  Loss: 0.943
Epoch: 3  Loss: 0.944
Epoch: 4  Loss: 0.944
Epoch: 5  Loss: 0.945
Epoch: 0  Loss: 6.269
Epoch: 1  Loss: 6.288
Epoch: 2  Loss: 6.308
Epoch: 3  Loss: 6.328
Epoch: 4  Loss: 6.348
Epoch: 5  Loss: 6.369
Epoch: 0  Loss: 1.657
Epoch: 1  Loss: 1.659
Epoch: 2  Loss: 1.662
Epoch: 3  Loss: 1.664
Epoch: 4  Loss: 1.666
Epoch: 5  Loss: 1.669
Epoch: 0  Loss: 2.243
Epoch: 1  Loss: 2.249
Epoch: 2  Loss: 2.256
Epoch: 3  Loss: 2.263
Epoch: 4  Loss: 2.271
Epoch: 5  Loss: 2.278
Epoch: 0  Loss: 0.870
Epoch: 1  Loss: 0.871
Epoch: 2  Loss: 0.871
Epoch: 3  Loss: 0.871
Epoch: 4  Loss: 0.872
Epoch: 5  Loss: 0.872
Epoch: 0  Loss: 3.715
Epoch: 1  Loss: 3.727
Epoch: 2  Loss: 3.739
Epoch: 3  

Epoch: 0  Loss: 2.445
Epoch: 1  Loss: 2.451
Epoch: 2  Loss: 2.457
Epoch: 3  Loss: 2.463
Epoch: 4  Loss: 2.469
Epoch: 5  Loss: 2.476
Epoch: 0  Loss: 4.692
Epoch: 1  Loss: 4.708
Epoch: 2  Loss: 4.724
Epoch: 3  Loss: 4.740
Epoch: 4  Loss: 4.756
Epoch: 5  Loss: 4.773
Epoch: 0  Loss: 3.585
Epoch: 1  Loss: 3.594
Epoch: 2  Loss: 3.603
Epoch: 3  Loss: 3.613
Epoch: 4  Loss: 3.622
Epoch: 5  Loss: 3.632
Epoch: 0  Loss: 1.760
Epoch: 1  Loss: 1.763
Epoch: 2  Loss: 1.767
Epoch: 3  Loss: 1.770
Epoch: 4  Loss: 1.773
Epoch: 5  Loss: 1.776
Epoch: 0  Loss: 2.769
Epoch: 1  Loss: 2.775
Epoch: 2  Loss: 2.781
Epoch: 3  Loss: 2.787
Epoch: 4  Loss: 2.793
Epoch: 5  Loss: 2.799
Epoch: 0  Loss: 1.204
Epoch: 1  Loss: 1.205
Epoch: 2  Loss: 1.207
Epoch: 3  Loss: 1.208
Epoch: 4  Loss: 1.210
Epoch: 5  Loss: 1.211
Epoch: 0  Loss: 1.806
Epoch: 1  Loss: 1.809
Epoch: 2  Loss: 1.812
Epoch: 3  Loss: 1.816
Epoch: 4  Loss: 1.819
Epoch: 5  Loss: 1.822
Epoch: 0  Loss: 2.303
Epoch: 1  Loss: 2.306
Epoch: 2  Loss: 2.309
Epoch: 3  

Epoch: 0  Loss: 1.974
Epoch: 1  Loss: 1.977
Epoch: 2  Loss: 1.980
Epoch: 3  Loss: 1.983
Epoch: 4  Loss: 1.986
Epoch: 5  Loss: 1.989
Epoch: 0  Loss: 0.961
Epoch: 1  Loss: 0.962
Epoch: 2  Loss: 0.963
Epoch: 3  Loss: 0.964
Epoch: 4  Loss: 0.965
Epoch: 5  Loss: 0.965
Epoch: 0  Loss: 2.279
Epoch: 1  Loss: 2.281
Epoch: 2  Loss: 2.284
Epoch: 3  Loss: 2.287
Epoch: 4  Loss: 2.289
Epoch: 5  Loss: 2.292
Epoch: 0  Loss: 1.864
Epoch: 1  Loss: 1.866
Epoch: 2  Loss: 1.869
Epoch: 3  Loss: 1.871
Epoch: 4  Loss: 1.873
Epoch: 5  Loss: 1.876
Epoch: 0  Loss: 0.951
Epoch: 1  Loss: 0.952
Epoch: 2  Loss: 0.952
Epoch: 3  Loss: 0.953
Epoch: 4  Loss: 0.953
Epoch: 5  Loss: 0.954
Epoch: 0  Loss: 2.262
Epoch: 1  Loss: 2.264
Epoch: 2  Loss: 2.267
Epoch: 3  Loss: 2.270
Epoch: 4  Loss: 2.273
Epoch: 5  Loss: 2.277
Epoch: 0  Loss: 1.095
Epoch: 1  Loss: 1.096
Epoch: 2  Loss: 1.098
Epoch: 3  Loss: 1.099
Epoch: 4  Loss: 1.100
Epoch: 5  Loss: 1.101
Epoch: 0  Loss: 21.031
Epoch: 1  Loss: 21.745
Epoch: 2  Loss: 22.525
Epoch: 

Epoch: 0  Loss: 2.092
Epoch: 1  Loss: 2.094
Epoch: 2  Loss: 2.097
Epoch: 3  Loss: 2.100
Epoch: 4  Loss: 2.103
Epoch: 5  Loss: 2.106
Epoch: 0  Loss: 1.376
Epoch: 1  Loss: 1.378
Epoch: 2  Loss: 1.380
Epoch: 3  Loss: 1.381
Epoch: 4  Loss: 1.383
Epoch: 5  Loss: 1.385
Epoch: 0  Loss: 4.594
Epoch: 1  Loss: 4.616
Epoch: 2  Loss: 4.638
Epoch: 3  Loss: 4.660
Epoch: 4  Loss: 4.683
Epoch: 5  Loss: 4.706
Epoch: 0  Loss: 5.112
Epoch: 1  Loss: 5.135
Epoch: 2  Loss: 5.159
Epoch: 3  Loss: 5.184
Epoch: 4  Loss: 5.209
Epoch: 5  Loss: 5.235
Epoch: 0  Loss: 1.868
Epoch: 1  Loss: 1.870
Epoch: 2  Loss: 1.872
Epoch: 3  Loss: 1.873
Epoch: 4  Loss: 1.875
Epoch: 5  Loss: 1.877
Epoch: 0  Loss: 12.534
Epoch: 1  Loss: 12.607
Epoch: 2  Loss: 12.682
Epoch: 3  Loss: 12.759
Epoch: 4  Loss: 12.839
Epoch: 5  Loss: 12.922
Epoch: 0  Loss: 2.624
Epoch: 1  Loss: 2.629
Epoch: 2  Loss: 2.634
Epoch: 3  Loss: 2.639
Epoch: 4  Loss: 2.644
Epoch: 5  Loss: 2.649
Epoch: 0  Loss: 6.508
Epoch: 1  Loss: 6.529
Epoch: 2  Loss: 6.551
Epoc

#### Make the predictions

In [ ]:
#Prepare the submission file for kaggle

schema={
 'FOODS_2': 'uint8', 'FOODS_3': 'uint8', 'HOBBIES_1': 'uint8', 'HOBBIES_2': 'uint8', 'HOUSEHOLD_1': 'uint8',
 'HOUSEHOLD_2': 'uint8', 'HOBBIES': 'uint8', 'HOUSEHOLD': 'uint8', 'TX': 'uint8', 'WI': 'uint8', 'CA_2': 'uint8',
 'CA_3': 'uint8','CA_4': 'uint8', 'TX_1': 'uint8', 'TX_2': 'uint8', 'TX_3': 'uint8', 'WI_1': 'uint8', 'WI_2': 'uint8',
 'WI_3': 'uint8', 'd': 'O', 'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int16', 'month': 'int16',
 'year': 'int16', 'snap_CA': 'int16', 'snap_TX': 'int16', 'snap_WI': 'int16', 'day': 'int16', 'Cinco De Mayo': 'int8',
 'OrthodoxChristmas': 'int8', "Father's day": 'int8', 'Thanksgiving': 'int8', "Mother's day": 'int8', 'NBAFinalsStart': 'int8',
 'Christmas': 'int8', 'Chanukah End': 'int8', 'EidAlAdha': 'int8', 'MemorialDay': 'int8', 'Halloween': 'int8',
 'VeteransDay': 'int8','SuperBowl': 'int8', 'LentStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
 'LaborDay': 'int8', 'Easter': 'int8', 'NBAFinalsEnd': 'int8', 'Purim End': 'int8', 'IndependenceDay': 'int8',
 'Ramadan starts': 'int8', 'StPatricksDay': 'int8', 'LentWeek2': 'int8', 'OrthodoxEaster': 'int8', 'Pesach End': 'int8',
 'NewYear': 'int8', 'PresidentsDay': 'int8', 'Eid al-Fitr': 'int8', 'ColumbusDay': 'int8', 'Sporting': 'int8',
 'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'sell_price': 'float16', 'last_month_avg': 'float16',
 'weekly_avg_4weeks_ago': 'float16'}


training_cols = ['id','item_id','day', 
                 'wm_yr_wk', 'year', 'FOODS_2', 'FOODS_3', 'HOBBIES_1',
       'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 'TX',
       'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2',
       'WI_3', 'snap_CA', 'snap_TX', 'snap_WI', 'Cinco De Mayo',
       'OrthodoxChristmas', 'Father\'s day', 'Thanksgiving', 'Mother\'s day',
       'NBAFinalsStart', 'Christmas', 'Chanukah End', 'EidAlAdha',
       'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart',
       'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter',
       'NBAFinalsEnd', 'Purim End', 'IndependenceDay', 'Ramadan starts',
       'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear',
       'PresidentsDay', 'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural',
       'National', 'Religious', 'wday', 'month', 'sell_price',
       'last_month_avg', 'weekly_avg_4weeks_ago', 'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]

#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

import pickle
import pandas as pd

with open('data/outputs/M5_Multiple_Linear_Regression_Baseline_Weights_Dictionary.pickle', 'rb') as f:
    weight_dict = pickle.load(f )
    
validation_df = pd.read_csv('../data/outputs/days_1913_1941__submission_validation.csv', dtype=schema)

    
#Prepare the submission file
validation_df['predictions'] = 0

for i in range(len(validation_df)):
    
    x = validation_df.iloc[i][input_cols]
    
    item = validation_df.loc[i, 'item_id']
    
    #dot product of the inputs with weights and additon of bias gives the prediction for each item
    validation_df.loc[i, 'predictions'] = x.dot(weight_dict[item][0]) + weight_dict[item][1][0]
    
kaggle_output_piv = validation_df.pivot_table(index='id', values='predictions', columns='day')
kaggle_output_piv.columns = submission_columns

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'submission_ML_MR_result_to_be_used_4submission.csv')
kaggle_output_piv

## MODEL 2: Random Forest Model


In [39]:
from sklearn.linear_model import SGDRegressor
import pickle
import pandas as pd
#################################################################################
schema={
 'FOODS_2': 'uint8', 'FOODS_3': 'uint8', 'HOBBIES_1': 'uint8', 'HOBBIES_2': 'uint8', 'HOUSEHOLD_1': 'uint8',
 'HOUSEHOLD_2': 'uint8', 'HOBBIES': 'uint8', 'HOUSEHOLD': 'uint8', 'TX': 'uint8', 'WI': 'uint8', 'CA_2': 'uint8',
 'CA_3': 'uint8','CA_4': 'uint8', 'TX_1': 'uint8', 'TX_2': 'uint8', 'TX_3': 'uint8', 'WI_1': 'uint8', 'WI_2': 'uint8',
 'WI_3': 'uint8', 'd': 'O', 'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int16', 'month': 'int16',
 'year': 'int16', 'snap_CA': 'int16', 'snap_TX': 'int16', 'snap_WI': 'int16', 'day': 'int16', 'Cinco De Mayo': 'int8',
 'OrthodoxChristmas': 'int8', "Father's day": 'int8', 'Thanksgiving': 'int8', "Mother's day": 'int8', 'NBAFinalsStart': 'int8',
 'Christmas': 'int8', 'Chanukah End': 'int8', 'EidAlAdha': 'int8', 'MemorialDay': 'int8', 'Halloween': 'int8',
 'VeteransDay': 'int8','SuperBowl': 'int8', 'LentStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
 'LaborDay': 'int8', 'Easter': 'int8', 'NBAFinalsEnd': 'int8', 'Purim End': 'int8', 'IndependenceDay': 'int8',
 'Ramadan starts': 'int8', 'StPatricksDay': 'int8', 'LentWeek2': 'int8', 'OrthodoxEaster': 'int8', 'Pesach End': 'int8',
 'NewYear': 'int8', 'PresidentsDay': 'int8', 'Eid al-Fitr': 'int8', 'ColumbusDay': 'int8', 'Sporting': 'int8',
 'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'sell_price': 'float16', 'last_month_avg': 'float16',
 'weekly_avg_4weeks_ago': 'float16'}


training_cols = ['id','item_id','day', 
                 'wm_yr_wk', 'year', 'FOODS_2', 'FOODS_3', 'HOBBIES_1',
       'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 'TX',
       'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2',
       'WI_3', 'snap_CA', 'snap_TX', 'snap_WI', 'Cinco De Mayo',
       'OrthodoxChristmas', 'Father\'s day', 'Thanksgiving', 'Mother\'s day',
       'NBAFinalsStart', 'Christmas', 'Chanukah End', 'EidAlAdha',
       'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart',
       'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter',
       'NBAFinalsEnd', 'Purim End', 'IndependenceDay', 'Ramadan starts',
       'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear',
       'PresidentsDay', 'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural',
       'National', 'Religious', 'wday', 'month', 'sell_price',
       'last_month_avg', 'weekly_avg_4weeks_ago', 'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]

#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

#################################################################################
 
training_df = pd.read_csv(export_path + final_preprocessed_file, usecols=training_cols, dtype=schema,chunksize=100000)
validation_df = pd.read_csv(export_path+validation_file, dtype=schema)
    
#Prepare the submission file
validation_df['predictions'] = 0

#initiate the model
model = SGDRegressor()

#train the model
for chunk in training_df:
    try:
        chunk.dropna(inplace=True)
        model.partial_fit(chunk[input_cols], chunk[target_col])
    except:
        continue
    
#Make predictions
validation_df['predictions'] = model.predict(validation_df[input_cols])

In [44]:
#Format the results for submission
kaggle_output_piv = validation_df.pivot_table(index='id', values='predictions', columns='day')
kaggle_output_piv.columns = submission_columns

#negative valuess should be zero and round  predictions
for col in kaggle_output_piv:
    kaggle_output_piv[kaggle_output_piv[col]<0]=0
kaggle_output_piv

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'submission_ML_SGD_result_to_be_used_4submission.csv')
kaggle_output_piv


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_evaluation,0.963142,0.863038,0.762935,0.662832,0.562728,0.993201,1.279227,0.889104,1.114656,0.653261,...,0.727874,1.486989,1.712541,1.251146,1.151043,1.050940,0.950836,0.850733,1.126487,1.026384
FOODS_1_001_CA_1_validation,0.963142,0.863038,0.762935,0.662832,0.562728,0.993201,1.279227,0.889104,1.114656,0.653261,...,0.727874,1.486989,1.712541,1.251146,1.151043,1.050940,0.950836,0.850733,1.126487,1.026384
FOODS_1_001_CA_2_evaluation,1.098952,0.998848,0.898745,0.798642,0.698538,1.129011,1.415037,1.024914,1.250466,0.789071,...,0.863684,1.622799,1.848351,1.386956,1.286853,1.186750,1.086646,0.986543,1.262297,1.162194
FOODS_1_001_CA_2_validation,1.098952,0.998848,0.898745,0.798642,0.698538,1.129011,1.415037,1.024914,1.250466,0.789071,...,0.863684,1.622799,1.848351,1.386956,1.286853,1.186750,1.086646,0.986543,1.262297,1.162194
FOODS_1_001_CA_3_evaluation,1.432300,1.332197,1.232093,1.131990,1.031887,1.462359,1.748385,1.358262,1.583814,1.122420,...,1.197033,1.956147,2.181699,1.720305,1.620201,1.520098,1.419995,1.319891,1.595645,1.495542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_WI_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
HOUSEHOLD_2_516_WI_2_evaluation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
HOUSEHOLD_2_516_WI_2_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
